In [1]:
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
time = "2024-12-31_13:33"
time = "2025-01-02_08:33"
time = "2025-01-03_06:00"
time = "2025-01-04_09:44"
time = "2025-01-06_17:52"
time = "2025-01-20_17:15"
time = "2025-01-20_18:26"
time = "2025-01-21_11:28" 


raw_df = pd.read_csv(f"/projects/genomic-ml/da2343/ml_project_2/unsupervised/kmeans/results/{time}_results.csv")
raw_df

,window,train_actual_return,train_num_trades,train_direction,test_actual_return,test_num_trades,test_direction,train_average_return,train_sharpe_ratio,train_profit_factor,...,total_windows,reverse_test,num_clusters,clustering_algorithm,train_period,test_period,random_seed,instrument,num_perceptually_important_points,price_history_length
0,301,0.077136,46,long,-0.002517,5,long,0.158392,1.750910,1000.0,...,101,False,7,birch,2,1,42,EXC_M15,4,24
1,302,0.097827,49,long,-0.009979,8,long,0.158392,1.750910,1000.0,...,101,False,7,birch,2,1,42,EXC_M15,4,24
2,303,0.090746,25,long,0.013855,2,long,0.158392,1.750910,1000.0,...,101,False,7,birch,2,1,42,EXC_M15,4,24
3,304,0.079172,24,long,-0.011884,3,long,0.158392,1.750910,1000.0,...,101,False,7,birch,2,1,42,EXC_M15,4,24
4,305,0.141315,26,long,-0.009990,3,long,0.158392,1.750910,1000.0,...,101,False,7,birch,2,1,42,EXC_M15,4,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4868,397,0.180887,63,long,0.000000,0,long,0.334177,1.459576,1000.0,...,101,False,9,birch,4,1,42,QCOM_M15,4,24
4869,398,0.142671,76,long,-0.019356,7,long,0.334177,1.459576,1000.0,...,101,False,9,birch,4,1,42,QCOM_M15,4,24
4870,399,0.188035,50,short,0.021674,5,short,0.334177,1.459576,1000.0,...,101,False,9,birch,4,1,42,QCOM_M15,4,24
4871,400,0.112546,25,long,-0.004083,1,long,0.334177,1.459576,1000.0,...,101,False,9,birch,4,1,42,QCOM_M15,4,24


In [9]:
def analyze_performance(raw_df):
    # Group by relevant parameters and calculate averages
    group_cols = ['instrument', 'num_clusters', 'clustering_algorithm', 'train_period', 
                 'reverse_test', 'random_seed']
    
    # Calculate mean performance metrics for each parameter combination
    results_df = raw_df.groupby(group_cols).agg({
        'test_profit_factor': 'mean',
        'test_sharpe_ratio': 'mean',
        'test_win_ratio': 'mean',
        'test_num_trades': 'mean'
    }).reset_index()
    
    # Calculate combined score
    results_df['combined_score'] = (
        results_df['test_profit_factor'] * 0.4 +
        results_df['test_sharpe_ratio'] * 0.4 +
        (results_df['test_num_trades'] / results_df['test_num_trades'].max()) * 0.2
    )
    
    # Create display DataFrame with proper formatting
    display_df = pd.DataFrame({
        'Instrument': results_df['instrument'],
        'Clusters': results_df['num_clusters'],
        'Algorithm': results_df['clustering_algorithm'],
        'Train Period': results_df['train_period'].astype(str) + 'w',
        'Rev.Test': results_df['reverse_test'],
        'Seed': results_df['random_seed'],
        'PF': results_df['test_profit_factor'].round(3),
        'Sharpe': results_df['test_sharpe_ratio'].round(3),
        'Win%': (results_df['test_win_ratio'] * 100).round(1),
        'Trades': results_df['test_num_trades'].round(1),
        'Score': results_df['combined_score'].round(3)
    })
    
    # Sort by instrument name
    display_df = display_df.sort_values('Instrument')
    
    # Apply styling with color gradients
    styled_df = display_df.style\
        .format({
            'PF': '{:.3f}',
            'Sharpe': '{:.3f}',
            'Win%': '{:.1f}%',
            'Trades': '{:.1f}',
            'Score': '{:.3f}'
        })\
        .background_gradient(subset=['Score'], cmap='YlOrRd')\
        .background_gradient(subset=['PF'], cmap='YlOrRd')\
        .background_gradient(subset=['Sharpe'], cmap='YlOrRd')\
        .background_gradient(subset=['Win%'], cmap='YlOrRd')\
        .background_gradient(subset=['Trades'], cmap='YlOrRd')\
        .set_properties(**{
            'text-align': 'right',
            'font-family': 'monospace',
            'padding': '5px'
        })\
        .hide(axis="index")
    
    return results_df, styled_df

# Example usage:
results_df, styled_df = analyze_performance(raw_df)
styled_df  # Display the styled table

Instrument,Clusters,Algorithm,Train Period,Rev.Test,Seed,PF,Sharpe,Win%,Trades,Score
AAPL_M15,7,kmeans,8w,False,622,0.798,-0.074,50.5%,3.5,0.423
ADI_M15,7,birch,3w,False,42,0.900,-0.029,53.5%,3.1,0.466
ADP_M15,7,birch,13w,False,42,1.395,0.083,47.8%,2.6,0.691
AMD_M15,9,kmeans,8w,False,622,0.502,-0.181,42.9%,2.8,0.234
AMGN_M15,5,birch,7w,False,42,1.239,0.068,50.0%,4.1,0.680
AMZN_M15,5,birch,8w,False,42,1.721,0.191,55.4%,4.6,0.942
ASML_M15,5,birch,4w,False,42,1.736,0.163,54.6%,4.1,0.917
ATVI_M15,9,kmeans,5w,False,300,2.121,0.144,49.5%,2.4,0.998
AVGO_M15,5,birch,1w,False,42,1.094,0.026,57.0%,4.2,0.607
BKNG_M15,5,birch,14w,False,42,0.710,-0.068,53.9%,3.7,0.399


In [10]:
results_df

,instrument,num_clusters,clustering_algorithm,train_period,reverse_test,random_seed,test_profit_factor,test_sharpe_ratio,test_win_ratio,test_num_trades,combined_score
0,AAPL_M15,7,kmeans,8,False,622,0.798029,-0.074398,0.505102,3.479592,0.422755
1,ADI_M15,7,birch,3,False,42,0.900132,-0.029051,0.534653,3.059406,0.465638
2,ADP_M15,7,birch,13,False,42,1.394777,0.082935,0.478495,2.612903,0.691185
3,AMD_M15,9,kmeans,8,False,622,0.502344,-0.181287,0.429293,2.767677,0.234452
4,AMGN_M15,5,birch,7,False,42,1.239283,0.068337,0.500000,4.091837,0.679806
5,AMZN_M15,5,birch,8,False,42,1.721413,0.191435,0.553763,4.623656,0.942271
6,ASML_M15,5,birch,4,False,42,1.736468,0.162505,0.545918,4.112245,0.917129
7,ATVI_M15,9,kmeans,5,False,300,2.120714,0.143667,0.495000,2.410000,0.998079
8,AVGO_M15,5,birch,1,False,42,1.094296,0.025819,0.570000,4.150000,0.607032
9,BKNG_M15,5,birch,14,False,42,0.709580,-0.068310,0.538889,3.711111,0.398680
